# Boosting
부스팅(Boosting)이란 단순하고 약한 학습기(Weak Learner)들를 결합해서 보다 정확하고 강력한 학습기(Strong Learner)를 만드는 방식.  
정확도가 낮은 하나의 모델을 만들어 학습 시킨뒤, 그 모델의 예측 오류는 두 번째 모델이 보완한다. 이 두 모델을 합치면 처음보다는 정확한 모델이 만들어 진다. 합쳐진 모델의 예측 오류는 다음 모델에서 보완하여 계속 더하는 과정을 반복한다.

- 약한 학습기들은 앞 학습기가 만든 오류를 줄이는 방향으로 학습한다.
- gradient boosting
    - 처음 모델은 y를 예측 두번째 부터는 앞 모델이 만든 오류를 예측 그것을 앞 모델에 업데이트하면 오류를 줄일 수 있다.
    - 그 오류를 update할 때 뺄까 더할까를 gradient descent 방법을 쓴다. 미분해서 나오는 값의 음수를 취해서 적용. 
    - 학습률을 작게 하면 update가 조금씩 크면 많이 하게 된다. 그래서 크게 하면 학습데이터에 너무 맞아 과대적합 될 수 있다.

## GradientBoosting
- 개별 모델로 Decision Tree 를 사용한다. 
- depth가 깊지 않은 트리를 많이 연결해서 이전 트리의 오차를 보정해 나가는 방식으로 실행한다.
- 오차를 보정할 때 경사하강법(Gradient descent)을 사용한다.
- 얕은 트리를 많이 연결하여 각각의 트리가 데이터의 일부에 대해 예측을 잘 수행하도록 하고 그런 트리들이 모여 전체 성능을 높이는 것이 기본 아이디어.
- 분류와 회귀 둘다 지원하는 모델 (GradientBoostingClassification, GrandientBoostingRegressor)
- 훈련시간이 많이 걸리고, 트리기반 모델의 특성 상 희소(0이 많은)한 고차원 데이터에서는 성능이 안 좋은 단점이 있다.

### 주요 파라미터
- Decision Tree 의 가지치기 관련 매개변수
    - 각각의 tree가 복잡한 모델이 되지 않도록 한다. 
- learning rate
    - 이전 tree의 오차를 얼마나 강하게 보정할 것인지 제어하는 값. 
    - 값이 크면 보정을 강하게 하여 복잡한 모델을 만든다. 학습데이터의 정확도는 올라가지만 과대적합이 날 수 있다. 
    - 값을 작게 잡으면 보정을 약하게 하여 모델의 복잡도를 줄인다. 과대적합을 줄일 수 있지만 성능 자체가 낮아질 수 있다.
    - 기본값 : 0.1 (로그스케일)
- n_estimators
    - tree의 개수 지정. 많을수록 복잡한 모델이 된다.
    - 최대한 크게 잡아준다.
- n_iter_no_change, validation_fraction
    - validation_fraction에 지정한 비율만큼 n_iter_no_change에 지정한 반복 횟수 동안 검증점수가 좋아지지 않으면 훈련을 조기 종료(early stopping)한다.

- 보통 max_depth를 낮춰 개별 트리의 복잡도를 낮춘다. (5가 넘지 않게) 그리고 n_estimators를 가용시간, 메모리 한도에 맞춘 뒤 적절한 learning_rate을 찾는다.




In [2]:
%%writefile util.py

# 평가지표 출력 함수
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def print_metrics(y, pred, title=None):
    acc = accuracy_score(y, pred)
    rec = recall_score(y, pred)
    pre = precision_score(y, pred)
    f1 = f1_score(y, pred)
    if title:
        print(title)
    print(f'정확도: {acc}, recall: {rec}, precision: {pre}, f1점수: {f1}')

Writing util.py


In [4]:
from util import print_metrics
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [5]:
data = load_breast_cancer()
X, y = data['data'], data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=0)
gb.fit(X_train, y_train)
pred_train = gb.predict(X_train)
pred_test = gb.predict(X_test)

print_metrics(y_train, pred_train)
print_metrics(y_test, pred_test)

정확도: 1.0, recall: 1.0, precision: 1.0, f1점수: 1.0
정확도: 0.958041958041958, recall: 0.9555555555555556, precision: 0.9772727272727273, f1점수: 0.9662921348314608


In [8]:
# decision tree 성능 낮추기 - max_depth를 작게 지정
gb = GradientBoostingClassifier(random_state=0, max_depth=1)   # learning rate = 0.1
gb.fit(X_train, y_train)
pred_train = gb.predict(X_train)
pred_test = gb.predict(X_test)

print_metrics(y_train, pred_train)
print_metrics(y_test, pred_test)

정확도: 0.9953051643192489, recall: 1.0, precision: 0.9925650557620818, f1점수: 0.9962686567164178
정확도: 0.965034965034965, recall: 0.9777777777777777, precision: 0.967032967032967, f1점수: 0.9723756906077348


In [11]:
# learning rate 낮추기
gb = GradientBoostingClassifier(random_state=0, max_depth=1, learning_rate = 0.001)   # 과소적합
gb.fit(X_train, y_train)
pred_train = gb.predict(X_train)
pred_test = gb.predict(X_test)

print_metrics(y_train, pred_train)
print_metrics(y_test, pred_test)

정확도: 0.6267605633802817, recall: 1.0, precision: 0.6267605633802817, f1점수: 0.7705627705627706
정확도: 0.6293706293706294, recall: 1.0, precision: 0.6293706293706294, f1점수: 0.7725321888412017


In [12]:
# 낮은 learning rate에 n_estimator 더 많이 주기
gb = GradientBoostingClassifier(random_state=0, max_depth=1, learning_rate = 0.001, n_estimators=500)
gb.fit(X_train, y_train)
pred_train = gb.predict(X_train)
pred_test = gb.predict(X_test)

print_metrics(y_train, pred_train)
print_metrics(y_test, pred_test)

정확도: 0.92018779342723, recall: 0.9925093632958801, precision: 0.8922558922558923, f1점수: 0.9397163120567377
정확도: 0.8951048951048951, recall: 0.9888888888888889, precision: 0.8640776699029126, f1점수: 0.9222797927461138


In [13]:
# feature 중요도 조회
gb.feature_importances_

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.15648382, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.1103002 , 0.        , 0.36581479, 0.05636868, 0.        ,
       0.        , 0.        , 0.3110325 , 0.        , 0.        ])

In [14]:
import pandas as pd
fi = pd.Series(gb.feature_importances_, index = data['feature_names'])
fi.sort_values(ascending=False)

worst perimeter            0.365815
worst concave points       0.311032
mean concave points        0.156484
worst radius               0.110300
worst area                 0.056369
mean radius                0.000000
concavity error            0.000000
worst symmetry             0.000000
worst concavity            0.000000
worst compactness          0.000000
worst smoothness           0.000000
worst texture              0.000000
fractal dimension error    0.000000
symmetry error             0.000000
concave points error       0.000000
compactness error          0.000000
mean texture               0.000000
smoothness error           0.000000
area error                 0.000000
perimeter error            0.000000
texture error              0.000000
radius error               0.000000
mean fractal dimension     0.000000
mean symmetry              0.000000
mean concavity             0.000000
mean compactness           0.000000
mean smoothness            0.000000
mean area                  0

### GridSearchCV 이용해 최적의 하이퍼파라미터 찾기

In [15]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    'n_estimators':range(500, 1001, 100),
    'learning_rate':[0.001, 0.05, 0.01, 0.1, 0.5],
    'max_depth':[1, 2, 3],
    'subsample':[0.5, 0.7, 0.9, 1]
}

rs = RandomizedSearchCV(GradientBoostingClassifier(random_state=0),
                       params,
                       cv=4,
                       n_iter=60, 
                       scoring='accuracy',
                       n_jobs=-1)

In [16]:
rs.fit(X_train, y_train)

RandomizedSearchCV(cv=4, estimator=GradientBoostingClassifier(random_state=0),
                   n_iter=60, n_jobs=-1,
                   param_distributions={'learning_rate': [0.001, 0.05, 0.01,
                                                          0.1, 0.5],
                                        'max_depth': [1, 2, 3],
                                        'n_estimators': range(500, 1001, 100),
                                        'subsample': [0.5, 0.7, 0.9, 1]},
                   scoring='accuracy')

In [18]:
result_df = pd.DataFrame(rs.cv_results_)
result_df.sort_values('rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
48,0.924866,0.006890,0.002242,0.000432,0.5,500,2,0.5,"{'subsample': 0.5, 'n_estimators': 500, 'max_d...",0.943925,0.971963,0.981132,0.990566,0.971896,0.017437,1
40,1.111701,0.016863,0.002613,0.000406,0.5,600,2,0.05,"{'subsample': 0.5, 'n_estimators': 600, 'max_d...",0.943925,0.962617,0.981132,0.990566,0.969560,0.017892,2
0,1.873853,0.014937,0.003242,0.000431,0.7,800,2,0.05,"{'subsample': 0.7, 'n_estimators': 800, 'max_d...",0.943925,0.971963,0.981132,0.981132,0.969538,0.015254,3
43,1.847941,0.012550,0.003250,0.001084,0.5,1000,2,0.1,"{'subsample': 0.5, 'n_estimators': 1000, 'max_...",0.943925,0.971963,0.981132,0.981132,0.969538,0.015254,3
3,2.496696,0.002405,0.003490,0.000501,0.7,800,3,0.1,"{'subsample': 0.7, 'n_estimators': 800, 'max_d...",0.943925,0.971963,0.981132,0.981132,0.969538,0.015254,3


In [19]:
rs.best_params_

{'subsample': 0.5, 'n_estimators': 500, 'max_depth': 2, 'learning_rate': 0.5}

In [20]:
rs.best_score_

0.9718964909187092

In [21]:
best_model = rs.best_estimator_
pred_test = best_model.predict(X_test)
print_metrics(y_test, pred_test)

정확도: 0.951048951048951, recall: 0.9777777777777777, precision: 0.946236559139785, f1점수: 0.9617486338797814


In [23]:
import numpy as np
fi = pd.Series(best_model.feature_importances_, index=data['feature_names'])
np.round(fi.sort_values(ascending=False), 2)

worst radius               0.50
worst concave points       0.23
mean concave points        0.05
worst perimeter            0.05
worst concavity            0.03
area error                 0.03
worst texture              0.03
radius error               0.02
mean area                  0.01
smoothness error           0.01
concavity error            0.01
worst fractal dimension    0.01
worst symmetry             0.01
symmetry error             0.00
worst smoothness           0.00
texture error              0.00
worst compactness          0.00
fractal dimension error    0.00
mean texture               0.00
worst area                 0.00
mean fractal dimension     0.00
compactness error          0.00
concave points error       0.00
mean symmetry              0.00
perimeter error            0.00
mean concavity             0.00
mean compactness           0.00
mean perimeter             0.00
mean smoothness            0.00
mean radius                0.00
dtype: float64

# XGBoost(Extra Gradient Boost)
- https://xgboost.readthedocs.io/
- Gradient Boost 알고리즘을 기반으로 개선해서 나온 모델.
- 캐글 경진대회에서 상위에 입상한 데이터 과학자들이 사용한 것으로 알려져 유명해짐.
- Gradient Boost의 단점인 느린 수행시간을 해결하고 과적합을 제어할 수 있는 규제를 제공하여 성능을 높임.
- 두 가지 개발 방법
    - [Scikit-learn 래퍼 XGBoost 모듈 사용](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)
    - [파이썬 래퍼 XGBoost 모듈 사용](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training)
- 설치
``
conda install -y -c anaconda py-xgboost
``

In [17]:
# !conda install -y -c anaconda py-xgboost

## Scikit-learn 래퍼 XGBoost
- XGBoost를 Scikit-learn프레임워크와 연동할 수 있도록 개발됨.
- Scikit-learn의 Estimator들과 동일한 패턴으로 코드를 작성할 수 있다.
- GridSearchCV나 Pipeline 등 Scikit-learn이 제공하는 다양한 유틸리티들을 사용할 수 있다.
- XGBClassifier: 분류
- XGBRegressor : 회귀 

### 주요 매개변수
- learning_rate : 학습률, 보통 0.01 ~ 0.2 사이의 값 사용
- n_estimators : week tree 개수
- max_depth: 트리의 depth 지정.

In [26]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=500, learning_rate=0.01)
xgb.fit(X_train, y_train)

C:\Users\Playdata\Anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:29:34] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [27]:
pred_train = xgb.predict(X_train)
pred_test = xgb.predict(X_test)
print_metrics(y_train, pred_train, "XGB train")
print_metrics(y_test, pred_test, "XGB test")

XGB train
정확도: 0.9976525821596244, recall: 1.0, precision: 0.996268656716418, f1점수: 0.9981308411214954
XGB test
정확도: 0.951048951048951, recall: 0.9555555555555556, precision: 0.9662921348314607, f1점수: 0.9608938547486034
